
# Manifold Modeling of Pentagon Spaces Using Laplacian Eigenfunctions

## Introduction :

A pentagon $p$ is a vector $(z_1,z_2,z_3,z_4,z_5)\in\mathbb{C}^5$ such that $d_i=|z_{i+1}-z_i|>0, \forall 1\leq i\leq 4$ where $z_6=z_1$.

Let $(d_1,d_2,d_3,d_4,d_5)\in\mathbb{R}^5$. The moduli space of pentagons with side lengths $(d_1,d_2,d_3,d_4,d_5)$ is the quotient 

$$\mathcal M_5(d_1,d_2,d_3,d_4,d_5):=\{p=(z_1,z_2,z_3,z_4,z_5)\in\mathbb{C}^5\mid d_i=|z_{i+1}-z_i|>0, \forall 1\leq i\leq 5,\}/\sim$$ 

where $z_6=z_1$. The equivalence relation $\sim$ is defined as follows: $p=(z_1,z_2,z_3,z_4,z_5),p'=(z'_1,z'_2,z'_3,z'_4,z'_5)$ are equivalent if and only if there exists $a,b\in C$ with $|a|=1$ such that $z_i=az'_i+b,\forall 1\leq i\leq 5$.

Klaus and Kojima prove that the space of pentagons is a closed smooth 2-manifold. However, depending on the choice of edge lengths, the moduli space will change topologically. Thus, the need to properly sample $\mathcal M_5(d_1,d_2,d_3,d_4,d_5)$ comes into play. Additionally, $\mathcal M_5(d_1,d_2,d_3,d_4,d_5) \subseteq \mathbb{C}^5$. For different moduli spaces, we know the surface they create: genus 3 surface or a sphere. These lie in higher dimensions and since we cannot view such high dimensional data directly, we use several tools to appriately reduce the dimension of our data and preserve topological data. 

These tools start with topological dimensionality reduction via DREiMac and then an unfolding onto a surfaces planar model. A final visualization uses the Moore-Penrose pseudo-inverse which models a best-fit manifold for our data.

## Sampling the Space of Pentagons

In [2]:
import numpy as np
import math

def get_intersections(x0, y0, r0, x1, y1, r1, n=10):#find intersection points of two circles
    # circle 1: (x0, y0), radius r0
    # circle 2: (x1, y1), radius r1

    d=math.sqrt((x1-x0)**2 + (y1-y0)**2)

    #change the following nones to be more descriptive

    # non intersecting
    if d > r0 + r1 :
        print('non intersecting')
        return 'non intersecting'
    # One circle within other
    if d < abs(r0-r1):
        print('inscribed')
        return 'inscribed'
    # coincident circles
    if d == 0 and r0 == r1:
        points=[]
        for i in range(math.ceil(n*0.01)):
            points.append(x0+r0*np.cos(i*2*math.pi*(1/math.ceil(n*0.01))),y0+r0*np.sin(i*2*math.pi*(1/math.ceil(n*0.01))))
        return points
    else:
        a=(r0**2-r1**2+d**2)/(2*d)
        h=math.sqrt(r0**2-a**2)
        x2=x0+a*(x1-x0)/d
        y2=y0+a*(y1-y0)/d
        x3=x2+h*(y1-y0)/d
        y3=y2-h*(x1-x0)/d

        x4=x2-h*(y1-y0)/d
        y4=y2+h*(x1-x0)/d

        return [[x3, y3], [x4, y4]]

In [3]:
def pentagon(z3, dist): #z3 in numpy array form
    if np.linalg.norm(z3-np.array([1,0]))>dist[0]+dist[1] or np.linalg.norm(z3)>dist[2]+dist[3]:
        print(z3)
        raise Exception('z3 out of range')

    z2s=get_intersections(z3[0], z3[1], dist[1], 1, 0, dist[0])
    z4s=get_intersections(z3[0], z3[1], dist[2], 0, 0, dist[3])

    output=[]
    for i in z2s:
        for j in z4s:
            if type(j)==str or type(i)==str:
                continue
            output.append([[1,0],i,z3,j,[0,0]])
            output.append([[1,0],[i[0],-1*i[1]],[z3[0],-1*z3[1]],[j[0],-1*j[1]],[0,0]])
    return np.asarray(output,dtype=object)    

In [4]:
def moduli_space_sample(n, dist):
    if dist[2]+dist[3]>=(1+dist[0]+dist[1]):
        x_vals=np.linspace(1-(dist[0]+dist[1]),1+dist[0]+dist[1],math.ceil(math.sqrt(n//2)))
        y_vals=np.linspace(-(dist[0]+dist[1]),dist[0]+dist[1],math.ceil(math.sqrt(n//2)))
    elif -(dist[2]+dist[3])>=1-(dist[0]+dist[1]):
        x_vals=np.linspace(-(dist[2]+dist[3]),dist[2]+dist[3],math.ceil(math.sqrt(n//2)))
        y_vals=np.linspace(-(dist[2]+dist[3]),dist[2]+dist[3],math.ceil(math.sqrt(n//2)))
    elif dist[2]+dist[3]<(1+dist[0]+dist[1]) or -(dist[2]+dist[3])<1-(dist[0]+dist[1]):
        x_vals=np.linspace(1-(dist[0]+dist[1]),dist[2]+dist[3],math.ceil(math.sqrt(n//2)))
        intersections=get_intersections(0,0,dist[2]+dist[3],1,0,dist[0]+dist[1])
        if 0<intersections[0][0]<1:
            ymax, ymin = max(intersections[0][1],intersections[1][1]), min(intersections[0][1],intersections[1][1])
        elif intersections[0][0]<=0:
            ymax, ymin = dist[2]+dist[3], -(dist[2]+dist[3])
        elif intersections[0][0]>=1:
            ymax, ymin = dist[0]+dist[1], -(dist[0]+dist[1])
        y_vals=np.linspace(ymin,ymax,math.ceil(math.sqrt(n//2)))

    coords=[np.array([x,y]) for x in x_vals for y in y_vals]

    output=[]
    for i in coords:
        if np.linalg.norm(i-np.array([1,0]))<=dist[0]+dist[1] and np.linalg.norm(i)<=dist[2]+dist[3]:
            for pent in pentagon(i,dist):
                output.append(pent)

    return np.asarray(output,dtype=object)

In [5]:
# import xlwt
# import xlrd
# from xlwt import Workbook

# r=moduli_space_sample(14000, [0.1,0.1,0.4,0.7])

# wb = Workbook()
# sheet1 = wb.add_sheet('Sheet 1')

# def clean(i):
#     return (str(np.array(i[0]).tolist())+', '+str(np.array(i[1]).tolist())).replace('[','').replace(']','')

# for count1, pent in enumerate(r):
#     for count2, i in enumerate(pent):
#         sheet1.write(count1, count2, clean(i))

# wb.save('data.xls')


# workbook = xlrd.open_workbook(r'data.xls')
# worksheet = workbook.sheet_by_name('Sheet 1')

# with open('data/data clean.txt', 'w+') as f:
#     for i in range(worksheet.nrows-1):
#         for j in range(4):
#             f.write(worksheet.cell(i, j).value)
#             f.write(', ')
#         f.write(worksheet.cell(i, 4).value)
#         f.write('\n')


## Example Surfaces:
The found edge lengths were taken from Klaus and Kojima's paper.

## Genus 4 Surface: $d = [1, 1, 1, 1, 1]$

## Sphere: $d = [0.4, 0.4, 0.4, 0.4, 1]$

## Genus 2 Surface: $d = [0.1, 0.1, 0.4, 0.7, 1]$

## Disjoint Tori: $d = [0.1, 0.1, 1, 1, 1]$

# Topological Dimensionality Reduction on Sampled Moduli Spaces

## Genus 4 Surface: $d = [1, 1, 1, 1, 1]$

## Sphere: $d = [0.4, 0.4, 0.4, 0.4, 0.4]$

# Unfolding the Surface of Pentagons

## DREiMac Torus

# Finding a Best-Fit Manifold Using Solutions to the Laplacian

## Sphere: $d = [0.4, 0.4, 0.4, 0.4, 0.4]$